# import data

In [92]:
import pandas as pd
import numpy as np

aseg_data = pd.read_csv('../dataset/final_data/aseg+DKT.stats980.csv')
aseg_data.head()

,Group,Age,Sex,Volume_mm30,normMean0,normStdDev0,normMin0,normMax0,Volume_mm31,normMean1,...,Volume_mm398,normMean98,normStdDev98,normMin98,normMax98,Volume_mm399,normMean99,normStdDev99,normMin99,normMax99
0,AD,78,M,239618.359,104.5263,8.7939,23.0,133.0,19019.868,21.8726,...,617.654,78.3675,12.5697,49.0,101.0,4935.454,70.7624,9.5176,42.0,95.0
1,AD,66,M,244462.620,104.4633,9.0946,19.0,135.0,52376.076,18.9785,...,866.656,81.6065,11.9081,49.0,106.0,5642.185,71.9476,9.2368,45.0,97.0
2,AD,77,M,236413.264,104.6093,8.0282,28.0,133.0,16591.926,27.7462,...,928.867,84.8252,10.9218,55.0,101.0,5959.229,76.6012,8.9234,50.0,99.0
3,AD,73,M,227601.449,104.3662,8.7423,29.0,132.0,19109.596,20.5128,...,576.647,80.3540,12.4069,51.0,105.0,5598.818,71.2126,8.7279,43.0,97.0
4,AD,62,M,220511.415,104.5355,7.3271,40.0,129.0,4690.350,36.5242,...,724.538,86.7110,10.1832,61.0,103.0,5327.887,76.5429,7.5031,51.0,96.0


In [93]:
aseg_X = aseg_data.iloc[:, 3:]
aseg_y = aseg_data.iloc[:, 0]
aseg_X = (aseg_X - aseg_X.mean()) / aseg_X.std()
aseg_X = aseg_X.fillna(0)
X, y = aseg_X.to_numpy(), aseg_y.to_numpy()

In [94]:
X[:5 ,:5]

array([[ 0.27810799,  0.29010181,  0.66922092, -0.72292249, -0.27528645],
       [ 0.44556863, -0.10696843,  1.09386788, -1.23435655, -0.04460606],
       [ 0.16731146,  0.81322609, -0.41209658, -0.08362991, -0.27528645],
       [-0.13730305, -0.71896082,  0.59635168,  0.04422861, -0.39062664],
       [-0.38239754,  0.34808667, -1.4021863 ,  1.45067228, -0.73664723]])

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# from tensorflow.keras.utils import to_categorical

y_onehot = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()
X_train, X_test, y_train, y_test, y_train_onehot, y_test_onehot = \
    train_test_split(X, y, y_onehot, test_size=0.2, random_state=42)

In [96]:
y_train.shape,  y_test.shape

((784,), (196,))

# SVC

## train and test

we use the grid search to find the best hyperparameters for the SVC.

In [40]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly','sigmoid'],
    'svc__random_state': [42]
}
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

In [41]:
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0.1, 1, 10, 100],
                         'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
                         'svc__random_state': [42]})

In [42]:
grid_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_svc__C', 'param_svc__kernel', 'param_svc__random_state', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [43]:
grid_search.best_score_

0.7275510204081632

In [44]:
grid_search.cv_results_['mean_test_score']

array([0.62755102, 0.48673469, 0.48979592, 0.48673469, 0.62346939,
       0.67959184, 0.60714286, 0.54795918, 0.62346939, 0.71020408,
       0.72653061, 0.48571429, 0.62346939, 0.71020408, 0.72755102,
       0.49285714])

In [45]:
grid_search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', SVC(C=100, kernel='poly', random_state=42))])

------------------

In [67]:
param_poly = {
    'svc__C': [0.1, 1, 10, 100, 1000],
    'svc__kernel': ['linear'],
}
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])
grid_search = GridSearchCV(pipeline, param_poly, cv=5, n_jobs=-1)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             n_jobs=-1,
             param_grid={'svc__C': [0.1, 1, 10, 100, 1000],
                         'svc__kernel': ['linear']})

In [69]:
grid_search.best_score_

0.6275510204081632

In [68]:
best_params = grid_search.best_params_
best_params

{'svc__C': 0.1, 'svc__kernel': 'linear'}

we use the best parameters of linear SVC finding above, to train the model on training data and analyse the parameters.

In [70]:
from sklearn.metrics import accuracy_score

svc = SVC(C=0.1, kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6224489795918368


In [87]:
svc.classes_, svc.coef_.shape

(array(['AD', 'CN', 'MCI'], dtype=object), (3, 500))

In [89]:
# find which attributes own largest weight on AD
w_attr = zip(svc.coef_[0], aseg_X.columns)
sorted(w_attr, key=lambda x: abs(x[0]), reverse=True)

[(-0.1662841498368754, 'Volume_mm312'),
 (-0.14776540291003337, 'normStdDev87'),
 (-0.14599158430077533, 'normMax73'),
 (-0.13845883666272807, 'Volume_mm341'),
 (0.1327477144900283, 'normMax16'),
 (0.13173491157180367, 'normMax11'),
 (-0.13006225115821396, 'normMax24'),
 (0.12985747891155924, 'normMax31'),
 (0.1296254031279354, 'normMax59'),
 (-0.127231284725164, 'Volume_mm351'),
 (-0.12497232891893774, 'Volume_mm327'),
 (0.1239827161528593, 'normMin0'),
 (-0.12200985341201695, 'normMin12'),
 (0.12063714375876768, 'normMean28'),
 (-0.12054150897538729, 'normMax92'),
 (0.1189593635713109, 'Volume_mm369'),
 (-0.1170276662942751, 'normStdDev54'),
 (-0.11583067477982993, 'normMean52'),
 (0.1156160993322546, 'normStdDev50'),
 (0.11405007825061017, 'normMin20'),
 (-0.11101516199833708, 'normMean71'),
 (0.11081350457132305, 'normMin30'),
 (0.11010052845184659, 'normStdDev44'),
 (0.10982208032258972, 'normMean13'),
 (0.10807582405298174, 'normMin3'),
 (0.10751441053089134, 'normStdDev27'),
 (-

# MLP


In [5]:
# change string label to integer
np.unique(y_train)

array(['AD', 'CN', 'MCI'], dtype=object)

In [6]:
def label_str2int(y):
    y_int  = np.where(y=='AD', 0, 
                np.where(y=='CN', 1, 
                    np.where(y=='MCI', 2, None)))
    return y_int.astype(int)

y_train_int = label_str2int(y_train)
y_test_int = label_str2int(y_test)
y_int = label_str2int(y)

In [7]:
y_train_int.shape, X_train.shape, y_test_int.shape, X_test.shape

((784,), (784, 500), (196,), (196, 500))

In [8]:
import tensorflow as tf

mlp_pipeline = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(1024, activation="relu", input_shape=(500,)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

mlp_pipeline.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)
mlp_pipeline.fit(
    X_train,
    y_train_int,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_int),
)

2024-11-01 10:37:38.204363: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 10:37:38.315108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 10:37:38.360842: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 10:37:38.374789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 10:37:38.469948: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/100


/home/zqy/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3209 - loss: 7473.2798 - val_accuracy: 0.2041 - val_loss: 877.6309
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3305 - loss: 1024.2373 - val_accuracy: 0.3316 - val_loss: 802.8583
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3219 - loss: 818.8314 - val_accuracy: 0.5459 - val_loss: 300.0367
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4195 - loss: 251.8438 - val_accuracy: 0.3469 - val_loss: 45.7254
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4178 - loss: 50.1758 - val_accuracy: 0.5357 - val_loss: 117.5420
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4416 - loss: 73.4560 - val_accuracy: 0.5510 - val_loss: 47.2311
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3762 - loss: 126.9730 - val_accuracy: 0.5459 - val_loss: 134.5070
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4773 - loss: 80.9970 - val_accu

In [148]:
np.min(mlp_pipeline.predict(X_test)), np.max(mlp_pipeline.predict(X_test))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


(1.1961978e-26, 1.0)

# GCN GAT

In [5]:
aseg_X

,Volume_mm30,normMean0,normStdDev0,normMin0,normMax0,Volume_mm31,normMean1,normStdDev1,normMin1,normMax1,...,Volume_mm398,normMean98,normStdDev98,normMin98,normMax98,Volume_mm399,normMean99,normStdDev99,normMin99,normMax99
0,0.278108,0.290102,0.669221,-0.722922,-0.275286,-0.327571,-0.894920,0.398582,-0.925806,-0.095635,...,-0.437133,-1.595250,1.284668,-1.508899,-0.678977,-0.688266,-1.671979,1.127013,-1.720394,-0.487246
1,0.445569,-0.106968,1.093868,-1.234357,-0.044606,2.246773,-1.621635,-1.359745,-1.480046,-0.788594,...,1.507072,-0.272659,0.526904,-1.508899,1.152055,0.319690,-1.124274,0.745040,-0.739979,-0.009267
2,0.167311,0.813226,-0.412097,-0.083630,-0.275286,-0.514953,0.579954,-0.488195,1.014032,-0.095635,...,1.992814,1.041644,-0.602756,0.406129,-0.678977,0.771865,1.026249,0.318721,0.894044,0.468713
3,-0.137303,-0.718961,0.596352,0.044229,-0.390627,-0.320646,-1.236368,0.276701,-1.480046,-1.308312,...,-0.757315,-0.784096,1.098205,-0.870556,0.785849,0.257839,-1.463932,0.052782,-1.393589,-0.009267
4,-0.382398,0.348087,-1.402186,1.450672,-0.736647,-1.433485,2.784130,2.365583,0.182673,0.770562,...,0.397416,1.811679,-1.448712,2.321157,0.053436,-0.128569,0.999308,-1.613316,1.220849,-0.248257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,0.061656,-0.325042,-0.253648,0.172087,0.416755,-0.983144,0.876632,0.269567,0.182673,-0.095635,...,2.018385,-0.422354,-1.442756,-0.551385,-0.678977,0.339646,-1.225848,-2.331148,0.240435,-1.921185
976,-0.089609,1.072897,0.079065,0.939238,-0.044606,-0.814449,-0.330040,1.547433,-1.202926,-0.268875,...,0.599495,-0.417862,-0.201997,0.086958,-0.678977,0.618592,-0.372358,-0.628049,0.240435,-0.248257
977,0.398850,-0.468113,-0.495134,-0.211488,-0.736647,-0.668052,1.442767,0.359450,1.291152,-0.095635,...,0.666011,1.463003,-0.569426,0.725301,0.785849,-0.534346,0.858407,-0.175477,1.547654,-0.009267
978,0.737125,-0.169995,-1.231734,-0.339347,-1.198008,0.014952,0.762079,-0.575504,1.845391,0.250844,...,-0.090419,0.714773,-0.590386,1.682815,0.053436,0.426695,0.502990,0.096040,0.240435,-0.248257


In [6]:
aseg_y.head(3)

0    AD
1    AD
2    AD
Name: Group, dtype: object

In [7]:
y_test.shape, type(y_test), type(X_test)

((196,), numpy.ndarray, numpy.ndarray)

## layer

In [97]:
# data
X_test[:2, :10]


array([[-1.15558045, -0.0420506 ,  0.151934  ,  0.8113797 , -0.39062664,
        -0.01744533, -0.13782128, -0.87707874,  0.45979255, -0.61535398],
       [-0.99102197, -1.10216511, -1.54947821,  1.45067228, -0.39062664,
        -1.14602775,  1.35746742,  0.555621  ,  1.01403204,  0.59732292]])

In [98]:
X_test_gnn = X_test.reshape(X_test.shape[0], -1, 5)
X_train_gnn = X_train.reshape(X_train.shape[0], -1, 5)
X_test_gnn[:2, :2, :]

array([[[-1.15558045, -0.0420506 ,  0.151934  ,  0.8113797 ,
         -0.39062664],
        [-0.01744533, -0.13782128, -0.87707874,  0.45979255,
         -0.61535398]],

       [[-0.99102197, -1.10216511, -1.54947821,  1.45067228,
         -0.39062664],
        [-1.14602775,  1.35746742,  0.555621  ,  1.01403204,
          0.59732292]]])

In [99]:
# shape: (n_samples, n_features, n_channels)
X_train_gnn.shape, X_test_gnn.shape

((784, 100, 5), (196, 100, 5))

In [20]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras as keras

class GCNLayer(layers.Layer):
    def __init__(self, output_dim):
        super(GCNLayer, self).__init__()
        self.output_dim = output_dim
        self.dense = keras.layers.Dense(self.output_dim)

    def build(self, input_shape):
        # Define a trainable weight matrix for the layer
        # W shape: [num_features, output_dim]

        # self.W = self.add_weight(shape=(input_shape[0][-1], self.output_dim),
        #                          initializer='random_normal', trainable=True)
        pass
    
    def call(self, inputs, adjacency_matrix):
        # inputs shape: [num_samples, num_nodes, num_features]
        # adjacency_matrix shape: [num_samples, num_nodes, num_nodes]
        # Dense layer
        # features shape: [n_samples, num_nodes, output_dim]
        features = self.dense(inputs)

        # features = tf.matmul(inputs, self.W)  

        # Normalize adjacency matrix (adding self-loops)
        A_hat = adjacency_matrix + tf.eye(tf.shape(adjacency_matrix)[1])  # Add self-loops
        D_hat_inv = tf.linalg.inv(tf.linalg.diag(tf.reduce_sum(A_hat, axis=-1)))  # Degree matrix inverse

        # Laplacian computation: output = D^(-1/2) * A * D^(-1/2) * features
        A_hat = tf.matmul(D_hat_inv, A_hat)
        A_hat = tf.matmul(A_hat, D_hat_inv)
        # A_hat shape: [n_samples, num_nodes, num_nodes]

        # Apply GCN operation
        # output shape: [n_samples, num_nodes, output_dim]

        # test ==============
        # A_hat = adjacency_matrix
        output = tf.matmul(A_hat, features)
        
        # activation
        output = tf.nn.relu(output)
        return output


In [9]:
import tensorflow as tf
from tensorflow.keras import layers

# SortPool only support sorting, we need to add a GCN layer with output_dim=1 before it
# that is to say, the 'num_features' of SortPool should be 1
class SortPool(layers.Layer):
    def __init__(self, output_dim):
        super(SortPool, self).__init__()
        self.output_dim = output_dim

    def call(self, inputs):
        # Inputs expected to be of shape [batch_size, num_nodes, num_features]
        if inputs.shape[2] != 1:
            raise ValueError("The 'num_features' of SortPool should be 1")
        inputs = tf.squeeze(inputs, axis=2)
        
        # Sort the features along the last axis (num_features)
        # sorted_features shape: [batch_size, num_nodes, num_features]
        sorted_features = tf.sort(inputs, axis=1, direction='ASCENDING')

        # Using the top 'output_dim' features after sorting
        if self.output_dim < tf.shape(sorted_features)[1]:
            # pooled_output shape: [batch_size, output_dim, num_features]
            pooled_output = sorted_features[:, :self.output_dim, :] 
        else:
            raise ValueError("dimension error")
        return pooled_output


In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# need to check========================
class ClusterPool(layers.Layer):
    def __init__(self, num_clusters):
        super(ClusterPool, self).__init__()
        self.num_clusters = num_clusters

    def call(self, inputs, cluster_assignments):
        # Inputs expected to be of shape [batch_size, num_nodes, num_features]
        # Cluster assignments should have shape [num_nodes]

        pooled_features = []
        
        for i in range(self.num_clusters):
            # Get the features of the nodes that belong to the current cluster
            cluster_indices = tf.where(cluster_assignments == i)
            cluster_indices = tf.squeeze(cluster_indices, axis=-1)
            
            # If there are nodes in the current cluster
            if tf.shape(cluster_indices)[0] > 0:
                # Gather the features corresponding to the cluster indices
                cluster_features = tf.gather(inputs, cluster_indices, axis=1)
                
                # Pooling operation (e.g., mean)
                pooled_feature = tf.reduce_mean(cluster_features, axis=1)  # [batch_size, num_features]
            else:
                pooled_feature = tf.zeros((tf.shape(inputs)[0], tf.shape(inputs)[-1]))  # Pad with zeros

            pooled_features.append(pooled_feature)

        # Stack all pooled features to form the output [batch_size, num_clusters, num_features]
        return tf.stack(pooled_features, axis=1)


In [11]:
import tensorflow as tf
from tensorflow.keras import layers

class GATLayer(layers.Layer):
    def __init__(self, output_dim, num_heads=1):
        super(GATLayer, self).__init__()
        self.output_dim = output_dim
        self.num_heads = num_heads

    def build(self, input_shape):
        # W shape: [num_features, output_dim]
        self.W = self.add_weight(shape=(input_shape[0][-1], self.output_dim),
                                 initializer='random_normal', trainable=True)
        # a shape: [2 * output_dim, 1]
        self.a = self.add_weight(shape=(2 * self.output_dim, 1),
                                 initializer='random_normal', trainable=True)

    def call(self, inputs, adjacency_matrix):
        # inputs shape: [num_samples, num_nodes, num_features]
        # Linear transformation
        h = tf.matmul(inputs, self.W)  # Shape: [batch_size, num_nodes, output_dim]

        # Attention mechanism
        # why it is a dot function?====================
        N = tf.shape(h)[1]  # Number of nodes
        a_input = tf.concat([tf.tile(tf.expand_dims(h, axis=2), [1, 1, N, 1]),
                             tf.tile(tf.expand_dims(h, axis=1), [1, N, 1, 1])], axis=-1)
        e = tf.nn.leaky_relu(tf.matmul(a_input, self.a))  # Shape: [batch_size, num_nodes, num_nodes, 1]
        e = tf.squeeze(e, axis=-1)  # Shape: [batch_size, num_nodes, num_nodes]

        # Masked attention
        zero_vec = -9e15 * tf.ones_like(e)
        attention = tf.where(adjacency_matrix > 0, e, zero_vec)
        attention = tf.nn.softmax(attention, axis=-1)  # Shape: [batch_size, num_nodes, num_nodes]

        # Apply attention to node features
        h_prime = tf.matmul(attention, h)  # Shape: [batch_size, num_nodes, output_dim]

        return tf.nn.elu(h_prime)

In [12]:
class AddNorm(layers.Layer):
    def __init__(self, type):
        super(AddNorm, self).__init__()
        if type == 'layer':
            # gat use layer
            self.norm = layers.LayerNormalization()
        elif type == 'batch':
            # gcn use batch
            self.norm = layers.BatchNormalization()
    def call(self, inputs, sublayer):
        # Apply residual connection followed by layer normalization
        return self.norm(inputs + sublayer)

## model

In [13]:
X_train_gnn.shape

(784, 100, 5)

In [113]:
# Define the GCN classification model
# no sort pooling and cluster pooling
# set hidden_units == num_features
class GCNClassifier1(tf.keras.Model):
    def __init__(self, hidden_units, num_classes, dropout_rate=0.5, num_blocks=2):
        super(GCNClassifier1, self).__init__()
        # hidden_units == num_features
        for i in range(num_blocks):
            setattr(self, f'gcn{i}', GCNLayer(hidden_units))
            setattr(self, f'drop{i}', layers.Dropout(dropout_rate))
            setattr(self, f'addnorm{i}', AddNorm('batch'))

        self.ffn = layers.Dense(1, activation='relu')
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, x, adjacency_matrix, num_blocks=2): 
        
        for i in range(num_blocks):
            y = getattr(self, f'gcn{i}')(x, adjacency_matrix)
            y = getattr(self, f'drop{i}')(y)
            if i == 0:
                x = y
            else:
                x = getattr(self, f'addnorm{i}') (x, y)

        # x shape: [batch_size, num_nodes, hidden_units]

        x = self.ffn(x)
        x = tf.reduce_mean(x, axis=2)
        x = self.dense(x)
        
        return x

inputs = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[2]))
a_matrix = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[1]))
num_blocks = 5
gcn = GCNClassifier1(hidden_units=16, num_classes=3, num_blocks=num_blocks)
outputs = gcn(inputs, a_matrix, num_blocks=num_blocks)
gcn_model = keras.Model(inputs=[inputs, a_matrix], outputs=outputs)
gcn_model.summary()

/home/zqy/venv/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'gcn_classifier1_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "functional_535"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_74      │ (None, 100, 5)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_75      │ (None, 100, 100)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gcn_classifier1_3   │ (None, 3)         │      1,760 │ input_layer_74[0… │
│ (GCNClassifier1)    │                   │            │ input_layer_75[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,760 (6.88 KB)

 Trainable params: 1,632 (6.38 KB)

 Non-trainable params: 128 (512.00 B)

In [77]:
class GCNClassifier2(tf.keras.Model):
    def __init__(self, hidden_units, num_classes, dropout_rate=0.5):
        super(GCNClassifier1, self).__init__()
        # hidden_units == num_features
        self.gcn1 = GCNLayer(hidden_units)
        self.drop1 = layers.Dropout(dropout_rate)
        self.norm1 = layers.BatchNormalization() # maybe layer normalization ???
        self.ffn = layers.Dense(1, activation='tanh')
        self.norm2 = layers.BatchNormalization()
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, x, adjacency_matrix, num_blocks=2): 
        for _ in range(num_blocks):
            x = self.gcn1(x, adjacency_matrix)
            x = self.drop1(x)+x
            x = self.norm1(x)
        x = self.ffn(x)
        x = self.norm2(x)
        x = self.dense(x)
        
        return x

## training

In [114]:
def get_adjacency_matrix(X):
    # Compute the correlation matrix
    for i in range(X.shape[0]):
        if i == 0:
            corr_matrix = np.corrcoef(X[i])
        else:
            corr_matrix = np.dstack((corr_matrix, np.corrcoef(X[i])))
    # corr_matrix shape: [num_samples, num_nodes, num_nodes]
    return corr_matrix

In [115]:
# adjacency_matrix
adjacency_matrix = (get_adjacency_matrix(X_train_gnn).mean(axis=2) > 0.5).astype(int)
adjacency_matrix[adjacency_matrix == 0] = 1e-10
A_train_gnn = np.tile(adjacency_matrix, (X_train_gnn.shape[0], 1, 1))
A_test_gnn = np.tile(adjacency_matrix, (X_test_gnn.shape[0], 1, 1))

In [118]:

gcn_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)


# Train the model

history = gcn_model.fit(
    [X_train_gnn, A_train_gnn],
    y_train_onehot,
    epochs=50,
    batch_size=22,
    validation_data=(
        [X_test_gnn, A_test_gnn],
        y_test_onehot,
    ),
)

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5662 - loss: 0.8358 - val_accuracy: 0.5663 - val_loss: 0.8804
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5879 - loss: 0.8389 - val_accuracy: 0.5612 - val_loss: 0.8889
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6107 - loss: 0.7876 - val_accuracy: 0.5459 - val_loss: 0.8932
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5784 - loss: 0.8357 - val_accuracy: 0.5612 - val_loss: 0.8760
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6075 - loss: 0.7954 - val_accuracy: 0.5561 - val_loss: 0.8695
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5950 - loss: 0.8012 - val_accuracy: 0.5663 - val_loss: 0.8711
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5951 - loss: 0.8111 - val_accuracy: 0.5765 - val_loss: 0.8814
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5760 - loss: 0.8553 - val_accuracy: 0.5561 - val_loss

## test

In [1]:
# callbacks
class SaveLayerOutputs(tf.keras.callbacks.Callback):
    def __init__(self, model, inputs, outputs_dir='/home/zqy/output/'):
        super(SaveLayerOutputs, self).__init__()
        # self.model = model
        self.inputs = inputs
        self.outputs_dir = outputs_dir
        self.layer_outputs = {}

    def on_epoch_end(self, epoch, logs=None):
        for layer in self.model.layers:
            intermediate_model = tf.keras.Model(inputs=self.model.input, outputs=layer.output)
            intermediate_output = intermediate_model.predict(self.inputs)
            self.layer_outputs[layer.name] = intermediate_output
            np.save(f'{self.outputs_dir}/epoch_{epoch+1}_{layer.name}.npy', intermediate_output)



NameError: name 'tf' is not defined

### GCN Classifier1

In [ ]:
# Define the GCN layers and the model directly
inputs = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[2]))
a_matrix = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[1]))

hidden_units = 16
num_classes = 3
dropout_rate = 0.5
num_blocks = 5

x = inputs
for i in range(num_blocks):
    gcn_layer = GCNLayer(hidden_units)
    drop_layer = layers.Dropout(dropout_rate)
    addnorm_layer = AddNorm('batch')
    
    y = gcn_layer(x, a_matrix)
    y = drop_layer(y)
    if i == 0:
        x = y
    else:
        x = addnorm_layer(x, y)

class ReduceMeanLayer(layers.Layer):
    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=2)

ffn = layers.Dense(1, activation='relu')
x = ffn(x)
x = ReduceMeanLayer()(x)
dense = layers.Dense(num_classes, activation='softmax')
outputs = dense(x)

gcn_model = keras.Model(inputs=[inputs, a_matrix], outputs=outputs)
gcn_model.summary()


### GCN Layer

In [102]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras as keras

inputs = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[2]))
a_matrix = keras.Input(shape=(X_train_gnn.shape[1], X_train_gnn.shape[1]))
output_dim = 16

dense = keras.layers.Dense(output_dim)
features = dense(inputs)

# Normalize adjacency matrix (adding self-loops)
A_hat = a_matrix + tf.eye(X_train_gnn.shape[1])  # Add self-loops
class ReduceMeanLayer(layers.Layer):
    def call(self, inputs):
        A_tmp = tf.reduce_sum(inputs, axis=-1)
        D_hat_inv  = tf.linalg.inv(tf.linalg.diag(A_tmp)) 
        A_hat_1 = inputs  # Initialize A_hat_1 with inputs
        A_hat_1 = tf.matmul(D_hat_inv, A_hat_1)
        A_hat_1 = tf.matmul(A_hat_1, D_hat_inv)
        return A_hat_1

A_hat = ReduceMeanLayer()(A_hat)

# Apply GCN operation
class OutputLayer(layers.Layer):
    def __init__(self):
        super(OutputLayer, self).__init__()
    def call(self, A_hat, features):
        return tf.matmul(A_hat, features)
output = OutputLayer()(A_hat, features)

# Activation
class ActivationLayer(layers.Layer):
    def __init__(self):
        super(ActivationLayer, self).__init__()
    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=-1)

output = ActivationLayer()(output)
output = layers.Dense(3, activation='softmax')(output)

model = keras.Model(inputs=[inputs, a_matrix], outputs=output)
model.summary()


Model: "functional_100"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_69      │ (None, 100, 100)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_21 (Add)        │ (None, 100, 100)  │          0 │ input_layer_69[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_68      │ (None, 100, 5)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_mean_layer_… │ (None, 100, 100)  │          0 │ add_21[0][0]      │
│ (ReduceMeanLayer)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_90 (Dense)    │ (None, 100, 16)   │         96 │ input_layer_68[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer_8      │ (None, 100, 16)   │          0 │ reduce_mean_laye… │
│ (OutputLayer)       │                   │            │ dense_90[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_layer_7  │ (None, 100)       │          0 │ output_layer_8[0… │
│ (ActivationLayer)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_91 (Dense)    │ (None, 3)         │        303 │ activation_layer… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 399 (1.56 KB)

 Trainable params: 399 (1.56 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:

# Compile the model
class SaveLayerOutputs(tf.keras.callbacks.Callback):
    def __init__(self, model, inputs, outputs_dir='/home/zqy/output1/'):
        super(SaveLayerOutputs, self).__init__()
        # self.model = model
        self.inputs = inputs
        self.outputs_dir = outputs_dir
        self.layer_outputs = {}

    def on_epoch_end(self, epoch, logs=None):
        for layer in self.model.layers:
            intermediate_model = tf.keras.Model(inputs=self.model.input, outputs=layer.output)
            intermediate_output = intermediate_model.predict(self.inputs)
            self.layer_outputs[layer.name] = intermediate_output
            np.save(f'{self.outputs_dir}/epoch_{epoch+1}_{layer.name}.npy', intermediate_output)



# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

# adjacency_matrix
adjacency_matrix = (get_adjacency_matrix(X_train_gnn).mean(axis=2) > 0.5).astype(int)
A_train_gnn = np.tile(adjacency_matrix, (X_train_gnn.shape[0], 1, 1))
A_test_gnn = np.tile(adjacency_matrix, (X_test_gnn.shape[0], 1, 1))

# Create the callback instance
save_layer_outputs = SaveLayerOutputs(model, [X_train_gnn, A_train_gnn])
save_layer_outputs.set_model(model)

# Train the model
history = model.fit(
    [X_train_gnn, A_train_gnn],
    y_train_onehot,
    epochs=2,
    batch_size=22,
    validation_data=(
        [X_test_gnn, A_test_gnn],
        y_test_onehot,
    ),
    callbacks=[save_layer_outputs]
)




/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Epoch 1/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step accuracy: 0.3610 - loss: 1.0960  
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.3652 - loss: 1.0955 - val_accuracy: 0.4388 - val_loss: 1.0822
Epoch 2/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4380 - loss: 1.0828
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4337 - loss: 1.0825 - val_accuracy: 0.5153 - val_loss: 1.0659


In [85]:
output_dir = '/home/zqy/output1/'
import os
import numpy as np
for file in os.listdir(output_dir):
    if file.endswith('.npy'):
        data = np.load(output_dir+file)
        if file.startswith('epoch_1_output_layer_7'):
            tmp = data
tmp

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [104]:
model.layers

[<InputLayer name=input_layer_69, built=True>,
 <InputLayer name=input_layer_68, built=True>,
 <ReduceMeanLayer name=reduce_mean_layer_16, built=True>,
 <Dense name=dense_90, built=True>,
 <OutputLayer name=output_layer_8, built=True>,
 <ActivationLayer name=activation_layer_7, built=True>,
 <Dense name=dense_91, built=True>]

In [101]:
np.isnan(X_train_gnn).sum().sum()

0